In [ ]:
import os
import gc
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# OHLCV Data for Bitcoin Perp Market with Technical Indicators
data = pd.read_csv("../data/BTCUSDT-binance-spot-1d-indicators.csv")